<a href="https://colab.research.google.com/github/Maleniski/matematicas_para_ciencia_datos/blob/main/Probabilidad_Estadistica/Analisis_LQ_ProjectoFinalProfGudelia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final.

**Nombre del proyecto:** Formulación matemática del Cociente de Localización (location quotient) para la estimación de concentración industrial.

**Alumno:** María Elena Martínez Manzanares

**Profesor:** Doctora Gudelia Figueroa Preciado

**Curso:** Matemáticas para Ciencia de Datos (Probabilidad y Estadística)

**Programa educativo:** Maestría en Ciencia de Datos de la Universidad de Sonora

Hermosillo, Sonora a 13 de noviembre del 2022


En el siguiente cuaderno se replican algunas de las cuentas planteadas en el artículo ["The location quotient as an estimator of industrial concentration"](https://linkinghub.elsevier.com/retrieve/pii/S0166046212000269). El artículo presenta una propuesta de formulación matemática para el cociente de locación utilizada por el U.S. Bureau of Labor Statistics con el objetivo de análizar la confiabilidad del mismo considerando diferentes giros industriales y segmentaciones del país.

Se van a replicar los cálculos propuestos en la Figura 1, Tabla 1, y algunos de la Tabla 2. 

Los cálculos fueron realizados a través de la base de datos County Business Patterns, en donde se puede acceder a la fuente original utilizada en el artículo en [esta](http://users.econ.umn.edu/~holmes/data/CBP/) pero se encuentran así mismo disponibles en [este](https://github.com/Maleniski/matematicas_para_ciencia_datos/tree/main/Probabilidad_Estadistica/FuenteDeDatos) repositorio.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import urllib.request
import os
from zipfile import ZipFile
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=031ac6d1816711c8f9184c945cb7d0aa539314826e92c1d2c6906405326820e6
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [ ]:
import wget

## Descarga de datos.

In [ ]:
NAICS_clasificacion = pd.read_csv("https://raw.githubusercontent.com/Maleniski/matematicas_para_ciencia_datos/main/Probabilidad_Estadistica/FuenteDeDatos/naics2000.csv", index_col=0)

wget.download("https://github.com/Maleniski/matematicas_para_ciencia_datos/raw/main/Probabilidad_Estadistica/FuenteDeDatos/est_cnty2000csv.zip")
with ZipFile("est_cnty2000csv.zip") as myzip:
    est_cnty2000 = myzip.open("est_cnty2000.csv")
est_cnty2000 = pd.read_csv(est_cnty2000, index_col=0)

wget.download("https://github.com/Maleniski/matematicas_para_ciencia_datos/raw/main/Probabilidad_Estadistica/FuenteDeDatos/est_zip2000csv.zip")
with ZipFile("est_zip2000csv.zip") as myzip:
    est_zip2000 = myzip.open("est_zip2000.csv")
est_zip2000 = pd.read_csv(est_zip2000, index_col=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
NAICS_clasificacion.reset_index(inplace=True)

In [ ]:
est_cnty2000.reset_index(inplace=True)

In [ ]:
est_zip2000.reset_index(inplace=True)

# Replica de la Figura 1.

Lo que propone los autores en esta figura es demostrar con datos concretos que el location quotient es sesgado si se supone que este se genera de un proceso de Poisson. 

Para ello, los autores mencionan que definen el location quotient ($LQ$) con rango teórico de $\{0,...,S\}$, donde $S\in\mathbb{N}\cup \{0\}$. No obstante, si se cambia el modelado de la variable al suponer que viene de un proceso de Poisson surgido del límite de un proceso Binomial con parámetros $S$, $\theta$, el rango se expande a $\{0,...,S, S+1, S+2,...\}$, lo cual genera que su esperanza se pueda descomponer en una suma de los primeros $S$ elementos y de los $S+1$ elementos hacia adelante de la siguiente manera.

$$
\begin{eqnarray*}
E(LQ)_0^{\infty}&=&E(LQ)_0^{S}+E(LQ)_{S+1}^{\infty},\\
                &=& \frac{1}{\lambda}\left(\sum_{s=0}^Ss\frac{\lambda^se^{-\lambda}}{s!}+\sum_{s=S+1}^{\infty}s\frac{\lambda^se^{-\lambda}}{s!}\right),\\
                &=& e^{-\lambda}\left(\sum_{k=0}^{S-1}\frac{\lambda^{k}}{k!}+\sum_{k=S+1}^{\infty}\frac{\lambda^k}{k!}\right).\qquad \textrm(1)
\end{eqnarray*}
$$

En un proceso de estimación a través de datos de un proceso de Poisson, el parámetro $\lambda$ es estimado a través de $S$ y $\theta$. Es decir, si suponemos $S$ suficientemente grande,

$$
\theta S \approx \lambda.
$$

En la práctica, si tenemos una cantidad de datos $S$ suficientemente grande, el cómputo de la primera sumatoria en (1) deberia proveer de un valor de la esperanza de $LQ$ con niveles bajos de sesgo y una buena aproximación de $\lambda$ a través de $\theta$ y $S$. Justamente esto es lo que es demostrado en la Tabla 1, donde se prueban diferentes valores de $\theta$ y $S$, y se analiza que tan próximo al valor 1 resulta el cálculo del primer sumando de (1). Es decir, se esta realizado un análisis del sesgo.

In [ ]:
import math

Se calculará la primera sumatoria de (1) con diferentes valores de $\theta$ y $S$.

$$
e^{-\lambda}\sum_{k=0}^{S-1}\frac{\lambda^{k}}{k!}.\qquad \textrm(2)
$$

In [ ]:
S=[2,5,10,50,100]
Theta=[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]

In [ ]:
suma = 0
Figura1Data = pd.DataFrame(columns = ['S', 'Theta', 'Esperanza'])
#Fijamos s
for s in S:
  #Fijamos theta
  for theta in Theta:
    #Calculamos (2)
    suma = 0 
    for k in list(range(0,s)):
      suma += (math.exp(-(s*theta))*(s*theta)**k)/math.factorial(k)
    Figura1Data = Figura1Data.append({'S' : str(s), 'Theta' : theta, 'Esperanza' : suma}, ignore_index = True)

In [ ]:
Figura1Data.head(2)

,S,Theta,Esperanza
0,2,0,1.000000
1,2,0.1,0.982477


Graficamos.

In [ ]:
fig = px.line(Figura1Data, x="Theta", y="Esperanza", color='S', symbol="S",
              labels={
                     "Esperanza": "E(LQ|s<S, Θ)",
                      "Theta":"Θ"
                 },
              title='Expected value of the LQ')
fig.show()

# Replica de Tabla 1

Se va a replicar la Tabla 1 que se presenta en el artículo. Las celdas de esta tabla contienen valores de $\bar{\lambda}$ que garantizan niveles de tolerancia $\hat{\tau}$ y nivel de confianza $(1-\alpha)$. La tabla se calculara haciendo un grid search de rango $\{1,2,3,...,400\}$ sobre $\bar{\lambda}$.

In [ ]:
from scipy.stats import poisson, binom

In [ ]:
Alpha=[0.1,0.05,0.01]
BarTolerance = [0.01,0.025,0.05]

In [ ]:
Tabla1Data = pd.DataFrame(columns = ['Tolerance', 'Alpha', 'BarLambda'])
for alpha in Alpha:
  for tolerance in BarTolerance:
    for BarLambda in list(range(0,400)):
      s_bar = poisson.ppf(1-alpha,BarLambda)
      if(poisson.pmf(s_bar,BarLambda)<tolerance):
        Tabla1Data = Tabla1Data.append({'Tolerance' : tolerance, 'Alpha' : alpha, 'BarLambda' : BarLambda}, ignore_index = True)
        break

In [ ]:
Tabla1Data

,Tolerance,Alpha,BarLambda
0,0.010,0.10,279.0
1,0.025,0.10,43.0
2,0.050,0.10,8.0
3,0.010,0.05,86.0
4,0.025,0.05,11.0
5,0.050,0.05,2.0
6,0.010,0.01,3.0
7,0.025,0.01,1.0
8,0.050,0.01,1.0


# Replica de Tabla 2

EL NAICS son siglas en inglés del Sistema de Clasificación Industrial de América del Norte. 

El NAICS se componen de seis dígitos que identifican los diferentes giros industriales que se encuentran en [Estados Unidos, Canadá y México](https://libguides.rice.edu/naicssiccodes#:~:text=North%20American%20Industry%20Classification%20System,1990's%20to%20replace%20SIC%20codes.) en donde (info. tomada de [aquí](https://www.bea.gov/help/faq/19)):

* los primeros dos dígitos designar sector industrial;
* el tercero subsector industrial;
* el cuarto designa grupo industrial;
* el quinto designa una identificación industrial de NAICS;
* el sexto designa un número de identificación nacional industrial.

Es posible encontrar un ejemplo en la [siguiente](https://libguides.csuchico.edu/c.php?g=414121&p=2821979) liga.

* 44 - Retail
* 445 - Food & Beverage Stores
* 4452 - Specialty Food Stores
* 445291 - Baked Goods Stores

En principio, cada registro de la tabla de NAICS agrupados por estados y ciudades debe contener códigos de NAICS de seis dígitos, pero existen contadas excepciones.

Los autores en la Tabla 2 realizan truncamientos de la columna "NAICS" de los registros de la tabla est_cnty2000, quedándose con los primeros dos, cuatro y seis dígitos, para posteriormente crear agregaciones a nivel código postal, ciudad, y estado.

Nosotros replicaremos su estudio a nivel estado considerando truncamientos de cuatro dígitos.

Para poder replicar lo realizado en la Tabla 2, debemos primero crear una aproximación de $\lambda$, $\bar{\lambda}$, de cada cada estado.

Si consideramos $\theta_j$ como la proporción (distribución empírica) de la cantidad de empresas que hay en una zona $j$, $x_j$, con el total de empresas, $x$, esta se calcula como

$$
\frac{x_j}{x} = \theta_j.
$$

Por otro lado, consideramos $s_k$ la cantidad total de industrias de giro $k$, entonces si estamos en el estado $j$, el estimador de $\lambda_k$ de la industria $k$, $\bar{\lambda}_k$, se estima de la siguiente manera

$$
\bar{\lambda}_k=\theta_j s_k.
$$

Por lo que, primero debemos calcular la distribución empírica de $\{\theta_j\}_{j}$, con $j$ siendo los estados por la NAICS.

Calcularemos primero la distribución empírica de cantidad de negocios en una región $j$, que en el artículo denotan por $\theta_j$.

In [ ]:
#Cantidad de negocios por state de Estados Unidos, serian las x_j
cantidad_neg_estados = est_cnty2000.groupby(["st"]).sum()["NUMBER"]
cantidad_neg_estados

st
1      99817
2      18501
4     114804
5      63185
6     799863
8     137528
9      92436
10     23771
11     19655
12    428438
13    200442
15     29853
16     37429
17    308067
18    146321
19     80890
20     74939
21     89921
22    101016
23     39466
24    128467
25    176222
26    236912
27    139080
28     59788
29    144755
30     31849
31     49623
32     48178
33     37414
34    233559
35     42782
36    492073
37    203903
38     20139
39    270509
40     85094
41    100645
42    294741
44     28534
45     97146
46     23783
47    130876
48    471509
49     55379
50     21564
51    175582
53    164018
54     41047
55    140415
56     18120
Name: NUMBER, dtype: int64

In [ ]:
#Cantidad total de negocios es la x
total_neg = est_cnty2000.groupby(["st"]).sum()["NUMBER"].sum()
total_neg

7070048

In [ ]:
#Calculamos theta_j
dest_empirica_negocios=cantidad_neg_estados / total_neg
dest_empirica_negocios

st
1     0.014118
2     0.002617
4     0.016238
5     0.008937
6     0.113134
8     0.019452
9     0.013074
10    0.003362
11    0.002780
12    0.060599
13    0.028351
15    0.004222
16    0.005294
17    0.043574
18    0.020696
19    0.011441
20    0.010600
21    0.012719
22    0.014288
23    0.005582
24    0.018171
25    0.024925
26    0.033509
27    0.019672
28    0.008457
29    0.020474
30    0.004505
31    0.007019
32    0.006814
33    0.005292
34    0.033035
35    0.006051
36    0.069600
37    0.028840
38    0.002848
39    0.038261
40    0.012036
41    0.014235
42    0.041689
44    0.004036
45    0.013741
46    0.003364
47    0.018511
48    0.066691
49    0.007833
50    0.003050
51    0.024835
53    0.023199
54    0.005806
55    0.019861
56    0.002563
Name: NUMBER, dtype: float64

In [ ]:
dest_empirica_negocios=dest_empirica_negocios.reset_index().rename(columns={"NUMBER":"DistribucionEmpirica"})

In [ ]:
dest_empirica_negocios.head(2)

,st,DistribucionEmpirica
0,1,0.014118
1,2,0.002617


Considerando NAICS de cuatro dígitos, comenzaremos truncando los seis dígitos en cuatro para lograr realizar los cálculos.

In [ ]:
est_cnty2000["NAICS_4digts"]=est_cnty2000["NAICS"].astype(str).str[:4]
est_cnty2000

,NAICS,st,county,EMPCAT,NUMBER,NAICS_4digts
0,113110,1,1,1,1,1131
1,113110,1,7,1,1,1131
2,113110,1,13,1,1,1131
3,113110,1,23,2,1,1131
4,113110,1,25,3,1,1131
...,...,...,...,...,...,...
1609347,99----,56,37,1,13,99--
1609348,99----,56,39,1,29,99--
1609349,99----,56,41,1,4,99--
1609350,99----,56,43,1,5,99--


Agrupamos por estados.

In [ ]:
state=est_cnty2000.groupby(["st","NAICS_4digts"]).sum().reset_index()[["st","NAICS_4digts", "NUMBER"]]
state

,st,NAICS_4digts,NUMBER
0,1,1131,43
1,1,1132,16
2,1,1133,882
3,1,1141,21
4,1,1142,6
...,...,...,...
14458,56,8133,46
14459,56,8134,131
14460,56,8139,188
14461,56,95--,43


Nos hace falta calcular $s_k$, es decir, la cantidad de negocios que hay de la industria $k$ a nivel nacional.

In [ ]:
s_k=state.groupby(["NAICS_4digts"]).sum()["NUMBER"]
s_k=s_k.reset_index().rename(columns={"NUMBER": "AgregacionPorIndustria"})
s_k

,NAICS_4digts,AgregacionPorIndustria
0,1131,469
1,1132,258
2,1133,12620
3,1141,2308
4,1142,363
...,...,...
287,8133,11057
288,8134,31628
289,8139,69010
290,95--,14793


Para hacer fácilmente los cálculos de $\bar{\lambda}$, ponemos la columna de AgregacionPorIndustria ($s_k$) y la DistribucionEmpirica ($\theta_j$) en la tabla state.

In [ ]:
state=state.merge(s_k, how="inner",on="NAICS_4digts")

In [ ]:
state=state.merge(dest_empirica_negocios, how="inner",on="st")

In [ ]:
state

,st,NAICS_4digts,NUMBER,AgregacionPorIndustria,DistribucionEmpirica
0,1,1131,43,469,0.014118
1,1,1132,16,258,0.014118
2,1,1133,882,12620,0.014118
3,1,1141,21,2308,0.014118
4,1,1142,6,363,0.014118
...,...,...,...,...,...
14458,11,8134,211,31628,0.002780
14459,11,8139,1317,69010,0.002780
14460,11,95--,58,14793,0.002780
14461,11,99--,342,98968,0.002780


Calculamos la estimación $\bar{\lambda}$.

In [ ]:
state["lambda_estimation"]=state["AgregacionPorIndustria"]*state["DistribucionEmpirica"]
state

,st,NAICS_4digts,NUMBER,AgregacionPorIndustria,DistribucionEmpirica,lambda_estimation
0,1,1131,43,469,0.014118,6.621479
1,1,1132,16,258,0.014118,3.642519
2,1,1133,882,12620,0.014118,178.172841
3,1,1141,21,2308,0.014118,32.585017
4,1,1142,6,363,0.014118,5.124940
...,...,...,...,...,...,...
14458,11,8134,211,31628,0.002780,87.927032
14459,11,8139,1317,69010,0.002780,191.850402
14460,11,95--,58,14793,0.002780,41.125098
14461,11,99--,342,98968,0.002780,275.134771


Los datos de la Tabla1Data son los que nos permiten determinar cuántos de los giros industriales de la tabla state tienen una estimación de lambda aceptable para considerar que el cociente de localización efectuada a partir de estos datos es poco sesgado. Para una fácil referencia, mostramos de nueva cuenta la tabla Tabla1Data.

In [ ]:
Tabla1Data

,Tolerance,Alpha,BarLambda
0,0.010,0.10,279.0
1,0.025,0.10,43.0
2,0.050,0.10,8.0
3,0.010,0.05,86.0
4,0.025,0.05,11.0
5,0.050,0.05,2.0
6,0.010,0.01,3.0
7,0.025,0.01,1.0
8,0.050,0.01,1.0


Finalmente ya podemos calcular cuantos giros de negocios (NAICS_4digts) de la tabla state, tienen estimaciones de lambda (lambda_estimation) mayores que los umbrales mínimos de la Tabla1Data (BarLambda).

In [ ]:
Tabla2Data = Tabla1Data.copy()

In [ ]:
Porcentaje_Con_Tolerancia=[]
for i in list(range(0,Tabla1Data.shape[0])):
  cantidad_empresas = state[state["lambda_estimation"]>Tabla1Data["BarLambda"].iloc[i]]["NUMBER"].sum()
  porcentaje_cant_empresas=(cantidad_empresas/total_neg)*100

  Porcentaje_Con_Tolerancia.append(porcentaje_cant_empresas)

In [ ]:
Tabla2Data["Porcentaje_Empresas_Con_Umbral"]=Porcentaje_Con_Tolerancia
Tabla2Data

,Tolerance,Alpha,BarLambda,Porcentaje_Empresas_Con_Umbral
0,0.010,0.10,279.0,89.617892
1,0.025,0.10,43.0,98.716091
2,0.050,0.10,8.0,99.891146
3,0.010,0.05,86.0,97.087587
4,0.025,0.05,11.0,99.820341
5,0.050,0.05,2.0,99.992730
6,0.010,0.01,3.0,99.982716
7,0.025,0.01,1.0,99.998289
8,0.050,0.01,1.0,99.998289


# Calculo de LQ

Por último, independientemente de la confiabilidad estadística del estimador, lo calculamos a continuación.

In [ ]:
state["LQ"]=state["NUMBER"]/state["lambda_estimation"]
state

,st,NAICS_4digts,NUMBER,AgregacionPorIndustria,DistribucionEmpirica,lambda_estimation,LQ
0,1,1131,43,469,0.014118,6.621479,6.494018
1,1,1132,16,258,0.014118,3.642519,4.392564
2,1,1133,882,12620,0.014118,178.172841,4.950249
3,1,1141,21,2308,0.014118,32.585017,0.644468
4,1,1142,6,363,0.014118,5.124940,1.170745
...,...,...,...,...,...,...,...
14458,11,8134,211,31628,0.002780,87.927032,2.399717
14459,11,8139,1317,69010,0.002780,191.850402,6.864724
14460,11,95--,58,14793,0.002780,41.125098,1.410331
14461,11,99--,342,98968,0.002780,275.134771,1.243027


In [ ]:
state["NAICS_4digts"] = state["NAICS_4digts"].astype(str)
state["NAICS_2digts"]=state["NAICS_4digts"].astype(str).str[:2]
state.head(2)

Podemos ver en las gráficas un comportamiento interesante para el número 81. Según las [clasificacion del NAICS](https://www.naics.com/six-digit-naics/?code=81) el se corresponde con otros servicios. Es debido a eso que (posiblemente) es de los giros a nivel país de mayor conteo.

In [ ]:
fig = px.scatter(state, x="lambda_estimation", y="LQ", color="NAICS_2digts", size="AgregacionPorIndustria",labels={
                     "lambda_estimation": "Estimación de lambda",
                      "LQ":"Cociente de localización (LQ)",
                      "NAICS_2digits": "NAICS de dos dígitos"
                 },
              title='Estimación de lambda contra cociente de localización', log_x=True, log_y=True)
fig.show()

In [ ]:
fig = px.scatter(state, x="lambda_estimation", y="LQ", color="NAICS_2digts",labels={
                     "lambda_estimation": "Estimación de lambda",
                      "LQ":"Cociente de localización (LQ)",
                      "NAICS_2digits": "NAICS de dos dígitos"
                 },
              title='Estimación de lambda contra cociente de localización',log_x=True, log_y=True)
fig.show()

In [ ]:
fig = px.scatter(state, x="lambda_estimation", y="NUMBER", color="NAICS_2digts",size="AgregacionPorIndustria",labels={
                     "lambda_estimation": "Estimación de lambda",
                      "LQ":"Cociente de localización (LQ)",
                      "NAICS_2digits": "NAICS de dos dígitos",
                      "NUMBER": "Cantidad de empresas del giro a nivel estado"
                 },
              title='Estimación de lambda contra cantidad de empresas a nivel estado segmentadas por giro', log_x=True, log_y=True)
fig.show()

In [ ]:
fig = px.scatter(state, x="lambda_estimation", y="NUMBER", color="NAICS_2digts",labels={
                     "lambda_estimation": "Estimación de lambda",
                      "LQ":"Cociente de localización (LQ)",
                      "NAICS_2digits": "NAICS de dos dígitos",
                      "NUMBER": "Cantidad de empresas del giro a nivel estado"
                 },
              title='Estimación de lambda contra cantidad de empresas a nivel estado segmentadas por giro', log_x=True, log_y=True)
fig.show()